In [16]:
import numpy as np
import pandas as pd

from imblearn.under_sampling import RandomUnderSampler
from sklearn.svm import SVC 
from sklearn.naive_bayes import GaussianNB 
from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler,StandardScaler
from imblearn.over_sampling import RandomOverSampler
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from xgboost import XGBClassifier

In [17]:
data = pd.read_csv("creditcard.csv")

data.head()

data['Class'].value_counts()

len(data['Class'])

print("percentage of fraudulent data instances: {}".format(data['Class'].value_counts()[1] *100 /len(data['Class'])))
print("percentage of normal data instances: {}".format(data['Class'].value_counts()[0] *100 /len(data['Class'])))

# Rescaling the data



rs = RobustScaler()

data['scaled_amount'] = rs.fit_transform(data['Amount'].values.reshape(-1, 1))
data['scaled_time'] = rs.fit_transform(data['Time'].values.reshape(-1, 1))

data.drop(['Amount', 'Time'], axis = 1, inplace = True)


# from sklearn.linear_model import LogisticRegression

X = data.drop(['Class'], axis = 1)
y = data['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0, 
                                                    stratify = y)

print(X_train.shape, y_train.shape)
print(y_train.value_counts())



percentage of fraudulent data instances: 0.1727485630620034
percentage of normal data instances: 99.827251436938
(227845, 30) (227845,)
0    227451
1       394
Name: Class, dtype: int64


In [25]:
all_results=[]
all_dataset=dict()

In [54]:
 def training_testing_function(X,y,name):
        
        #removing highly correlated features
        corr = X.corr()
        columns = np.full((corr.shape[0],), True, dtype=bool)
        for i in range(corr.shape[0]):
            for j in range(i+1, corr.shape[0]):
                if corr.iloc[i,j] >= 0.8:
                    if columns[j]:
                        columns[j] = False
        selected_columns = X.columns[columns]
        X1 = X[selected_columns]
        
         # Train KNeighborsClassifier Model
#         KNN_Classifier = KNeighborsClassifier(n_jobs=-1)
        
        # Train LogisticRegression Model
        LGR_Classifier = LogisticRegression(multi_class='auto', random_state=1,solver='lbfgs',max_iter=400)

        # Train Gaussian Naive Baye Model
        GNB_Classifier = GaussianNB()

        # Train Decision Tree Model
        DTC_Classifier = tree.DecisionTreeClassifier(criterion='entropy', random_state=0)
        # DTC_Classifier.fit(X_train_random_oversampled, y_train_random_oversampled.values.ravel());
        
        XGB_Classifier = XGBClassifier(scale_pos_weight=500)
        

        ## Train Ensemble Model (This method combines all the individual models above except RandomForest)
        combined_model = [('Naive Baye Classifier', GNB_Classifier), 
                         ('Decision Tree Classifier', DTC_Classifier), 
#                          ('KNeighborsClassifier', KNN_Classifier), 
                         ('LogisticRegression', LGR_Classifier),
                          ('Gradient Boosting Classifier', XGB_Classifier)
                        ]
        VC =  VotingClassifier(estimators = combined_model,voting = 'soft')
        VC.fit(X1, y.values.ravel());

        models = []
        models.append(('VotingClassifier', VC))

        for i, v in models:
            accuracy = metrics.accuracy_score(y_test.values.ravel(), v.predict(X_test[selected_columns]))
            f1 = metrics.f1_score(y_test.values.ravel(),v.predict(X_test[selected_columns]))
            auc_score = metrics.roc_auc_score(y_test.values.ravel(),v.predict(X_test[selected_columns]))
            confusion_matrix = metrics.confusion_matrix(y_test.values.ravel(), v.predict(X_test[selected_columns]))
            classification = metrics.classification_report(y_test.values.ravel(), v.predict(X_test[selected_columns]))

            
            all_results.append((name+str(" RCF"),accuracy,f1,auc_score,confusion_matrix,classification)) #RCF = Removing correlated features
            print("{} Completed".format(name + str(" RCF")))
        
        
        
        
      

        rfc = RandomForestClassifier();

        # fit random forest classifier on the training set
        rfc.fit(X, y.values.ravel());
        # extract important features
        score = np.round(rfc.feature_importances_,3)

        importances = pd.DataFrame({'feature':X.columns,'importance':score})
        importances = importances.sort_values('importance',ascending=False).set_index('feature')

        min_features=min(len(importances),20)
        
        final_features=list(importances[:20].index)

        X2=X[final_features]

        # Train KNeighborsClassifier Model
#         KNN_Classifier = KNeighborsClassifier(n_jobs=-1)
        # KNN_Classifier.fit(X_train_random_oversampled, y_train_random_oversampled.values.ravel()); 

        # Train LogisticRegression Model
        LGR_Classifier = LogisticRegression(multi_class='auto', random_state=1,solver='lbfgs',max_iter=400)
        # LGR_Classifier.fit(X_train_random_oversampled, y_train_random_oversampled.values.ravel());

        # Train Gaussian Naive Baye Model
        GNB_Classifier = GaussianNB()
        # GNB_Classifier.fit(X_train_random_oversampled, y_train_random_oversampled.values.ravel())

        # Train Decision Tree Model
        DTC_Classifier = tree.DecisionTreeClassifier(criterion='entropy', random_state=0)
        # DTC_Classifier.fit(X_train_random_oversampled, y_train_random_oversampled.values.ravel());
        
        XGB_Classifier = XGBClassifier(scale_pos_weight=500)

        ## Train Ensemble Model (This method combines all the individual models above except RandomForest)
        combined_model = [('Naive Baye Classifier', GNB_Classifier), 
                         ('Decision Tree Classifier', DTC_Classifier), 
#                          ('KNeighborsClassifier', KNN_Classifier), 
                         ('LogisticRegression', LGR_Classifier),
                        ('Gradient Boosting Classifier', XGB_Classifier)
                        ]
        VC =  VotingClassifier(estimators = combined_model,voting = 'soft')
        VC.fit(X2, y.values.ravel());

        models = []

        models.append(('VotingClassifier', VC))

        for i, v in models:
            accuracy = metrics.accuracy_score(y_test.values.ravel(), v.predict(X_test[final_features]))
            confusion_matrix = metrics.confusion_matrix(y_test.values.ravel(), v.predict(X_test[final_features]))
            classification = metrics.classification_report(y_test.values.ravel(), v.predict(X_test[final_features]))
            f1 = metrics.f1_score(y_test.values.ravel(),v.predict(X_test[final_features]))
            auc_score = metrics.roc_auc_score(y_test.values.ravel(),v.predict(X_test[final_features]))
        
            all_results.append((name+str(" Random Forest  Selection"),accuracy,f1,auc_score,confusion_matrix,classification))
            print("{} Completed".format(name+str(" Random Forest Selection")))
    #         print('============================== {} Model Test Results =============================='.format(i))
    #         print()
    #         print ("Model Accuracy:" "\n", accuracy)
    #         print()
    #         print("Confusion matrix:" "\n", confusion_matrix)
    #         print()
    #         print("Classification report:" "\n", classification) 
    #         print()     
        
#         clf = RandomForestClassifier(n_estimators=100, n_jobs=-1)

#         # Build step forward feature selection
#         sfs1 = sfs(clf,
#            k_features=5,
#            forward=True,
#            floating=False,
#            verbose=2,
#            scoring='f1',
#            cv=5)

#         # Perform SFFS
#         sfs1 = sfs1.fit(X, y.values.ravel())
        
#         feat_cols = np.array(sfs1.k_feature_idx_) - 1
        fs = SelectKBest(score_func=f_classif, k=5)
        # apply feature selection
        fs.fit(X, y)
        X3 = fs.transform(X)
        X3_test = fs.transform(X_test)

                  
     # Train KNeighborsClassifier Model
#         KNN_Classifier = KNeighborsClassifier(n_jobs=-1)
        # KNN_Classifier.fit(X_train_random_oversampled, y_train_random_oversampled.values.ravel()); 

        # Train LogisticRegression Model
        LGR_Classifier = LogisticRegression(multi_class='auto', random_state=1,solver='lbfgs',max_iter=400)
        # LGR_Classifier.fit(X_train_random_oversampled, y_train_random_oversampled.values.ravel());

        # Train Gaussian Naive Baye Model
        GNB_Classifier = GaussianNB()
        # GNB_Classifier.fit(X_train_random_oversampled, y_train_random_oversampled.values.ravel())

        # Train Decision Tree Model
        DTC_Classifier = tree.DecisionTreeClassifier(criterion='entropy', random_state=0)
        # DTC_Classifier.fit(X_train_random_oversampled, y_train_random_oversampled.values.ravel());
        
        XGB_Classifier = XGBClassifier(scale_pos_weight=500)

        ## Train Ensemble Model (This method combines all the individual models above except RandomForest)
        combined_model = [('Naive Baye Classifier', GNB_Classifier), 
                         ('Decision Tree Classifier', DTC_Classifier), 
#                          ('KNeighborsClassifier', KNN_Classifier), 
                         ('LogisticRegression', LGR_Classifier),
                         ('Gradient Boosting Classifier', XGB_Classifier)
                        ]
        VC =  VotingClassifier(estimators = combined_model,voting = 'soft')
        VC.fit(X3, y.values.ravel());

        models = []

        models.append(('VotingClassifier', VC))

        for i, v in models:
            accuracy = metrics.accuracy_score(y_test.values.ravel(), v.predict(X3_test))
            confusion_matrix = metrics.confusion_matrix(y_test.values.ravel(), v.predict(X3_test))
            classification = metrics.classification_report(y_test.values.ravel(), v.predict(X3_test))
            f1 = metrics.f1_score(y_test.values.ravel(),v.predict(X3_test))
            auc_score = metrics.roc_auc_score(y_test.values.ravel(),v.predict(X3_test))
        
            all_results.append((name+str(" Forward feature selection"),accuracy,f1,auc_score,confusion_matrix,classification))
            print("{} Completed".format(name+str(" Forward feature selection")))
    #         print('============================== {} Model Test Results =============================='.format(i))
    #         print()
    #         print ("Model Accuracy:" "\n", accuracy)
    #         print()
    #         print("Confusion matrix:" "\n", confusion_matrix)
    #         print()
    #         print("Classification report:" "\n", classification) 
    #         print()     
            






#### 1. Random Undersample

In [27]:
undersample = RandomUnderSampler(sampling_strategy='majority')
# fit and apply the transform
all_dataset["X_train_undersample"], all_dataset["y_train_undersample"] = undersample.fit_resample(X_train, y_train)
# summarize class distribution
print(Counter(all_dataset["y_train_undersample"]))

Counter({0: 394, 1: 394})


In [28]:
training_testing_function(all_dataset["X_train_undersample"],all_dataset["y_train_undersample"],"Random Undersample")

Random Undersample RCF Completed
Random Undersample Random Forest Selection Completed
Random Undersample Forward feature selection Completed


#### 2. Instance Hardness Threshold

In [30]:
from imblearn.under_sampling import InstanceHardnessThreshold

In [31]:
iht = InstanceHardnessThreshold(sampling_strategy='majority', random_state=42)
all_dataset["X_train_res"], all_dataset["y_train_res"] = iht.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(all_dataset["y_train_res"]))

Resampled dataset shape Counter({0: 222288, 1: 394})


In [32]:
training_testing_function(all_dataset["X_train_res"],all_dataset["y_train_res"],"IHT")

IHT RCF Completed
IHT Random Forest Selection Completed
IHT Forward feature selection Completed


#### 3. Cluster Centroid

In [38]:
from imblearn.under_sampling import ClusterCentroids

In [39]:
all_dataset["X_cluster_centroids"],all_dataset["y_cluster_centroids"]  = ClusterCentroids().fit_resample(X_train,y_train)

In [40]:
training_testing_function(all_dataset["X_cluster_centroids"],all_dataset["y_cluster_centroids"],"Cluster centroids")

Cluster centroids RCF Completed
Cluster centroids Random Forest Selection Completed
Cluster centroids Forward feature selection Completed


In [41]:
all_results[-1]

('Cluster centroids Forward feature selection',
 0.9641515396229065,
 0.07851985559566788,
 0.9260191520333514,
 array([[54833,  2031],
        [   11,    87]], dtype=int64),
 '              precision    recall  f1-score   support\n\n           0       1.00      0.96      0.98     56864\n           1       0.04      0.89      0.08        98\n\n    accuracy                           0.96     56962\n   macro avg       0.52      0.93      0.53     56962\nweighted avg       1.00      0.96      0.98     56962\n')

In [8]:
all_results

[('Cluster centroids RCF',
  0.9212984094659598,
  0.036949516648764766,
  0.8994624122862425),
 ('Cluster centroids Random Forest  Selection',
  0.9923106632491837,
  0.2819672131147541,
  0.9350297308005926),
 ('Cluster centroids Forward feature selection',
  0.9918015519118009,
  0.26917057902973396,
  0.9347747364280545)]

#### 4. Near Miss

In [42]:
from imblearn.under_sampling import NearMiss

In [43]:
all_dataset['X_near_miss'],all_dataset['y_near_miss'] = NearMiss().fit_resample(X_train,y_train)
training_testing_function(all_dataset['X_near_miss'],all_dataset['y_near_miss'],"Near Miss")

Near Miss RCF Completed
Near Miss Random Forest Selection Completed
Near Miss Forward feature selection Completed


#### 5. One Sided Selection

In [45]:
from imblearn.under_sampling import OneSidedSelection

In [46]:
all_dataset['X_one_sided_selection'],all_dataset['y_one_sided_selection'] = OneSidedSelection().fit_resample(X_train,y_train)
training_testing_function(all_dataset['X_one_sided_selection'],all_dataset['y_one_sided_selection'],"One Sided Selection")

One Sided Selection RCF Completed
One Sided Selection Random Forest Selection Completed
One Sided Selection Forward feature selection Completed


In [14]:
all_results

[('Cluster centroids RCF',
  0.9212984094659598,
  0.036949516648764766,
  0.8994624122862425),
 ('Cluster centroids Random Forest  Selection',
  0.9923106632491837,
  0.2819672131147541,
  0.9350297308005926),
 ('Cluster centroids Forward feature selection',
  0.9918015519118009,
  0.26917057902973396,
  0.9347747364280545),
 ('Near Miss RCF',
  0.5980653769179453,
  0.007456539645380847,
  0.7375673644528155),
 ('Near Miss Random Forest  Selection',
  0.6635300726800323,
  0.008996897621509825,
  0.7754493715043699),
 ('Near Miss Forward feature selection',
  0.6825778589234929,
  0.009857072449482503,
  0.8002694219218356),
 ('One Sided Selection RCF',
  0.9993679997191109,
  0.8085106382978723,
  0.8876320013092462),
 ('One Sided Selection Random Forest  Selection',
  0.9993153330290369,
  0.7914438502673796,
  0.8774191267671952),
 ('One Sided Selection Forward feature selection',
  0.9993679997191109,
  0.8064516129032258,
  0.8825387534023176)]

#### 6. Tomek Links

In [47]:
from imblearn.under_sampling import TomekLinks

In [48]:
all_dataset['X_tomek_links'],all_dataset['y_tomek_links'] = TomekLinks().fit_resample(X_train,y_train)


In [49]:
training_testing_function(all_dataset['X_tomek_links'],all_dataset['y_tomek_links'],"Tomek Links")

Tomek Links RCF Completed
Tomek Links Random Forest Selection Completed
Tomek Links Forward feature selection Completed


### Oversampling data

#### 1. Random Oversampling

In [50]:
ros = RandomOverSampler(random_state=0)
all_dataset["X_train_random_oversampled"], all_dataset["y_train_random_oversampled"] = ros.fit_resample(X_train, y_train)
from collections import Counter
print(sorted(Counter(all_dataset["y_train_random_oversampled"]).items()))

[(0, 227451), (1, 227451)]


In [51]:
training_testing_function(all_dataset["X_train_random_oversampled"],all_dataset["y_train_random_oversampled"],"Random Oversampled")

Random Oversampled RCF Completed
Random Oversampled Random Forest Selection Completed
Random Oversampled Forward feature selection Completed


#### 2. SMOTE

In [52]:
from imblearn.over_sampling import SMOTE, ADASYN
all_dataset["X_smote"], all_dataset["y_smote"] = SMOTE().fit_resample(X_train, y_train)
print(sorted(Counter(all_dataset["y_smote"]).items()))



[(0, 227451), (1, 227451)]


In [55]:
training_testing_function(all_dataset['X_smote'],all_dataset['y_smote'],"SMOTE")

SMOTE RCF Completed
SMOTE Random Forest Selection Completed
SMOTE Forward feature selection Completed


#### 3. ADASYN

In [56]:
all_dataset['X_adasyn'],all_dataset['y_adasyn'] = ADASYN().fit_resample(X_train, y_train)
print(sorted(Counter(all_dataset["y_adasyn"]).items()))

[(0, 227451), (1, 227448)]


In [57]:
training_testing_function(all_dataset['X_adasyn'],all_dataset['y_adasyn'],"ADASYN")

ADASYN RCF Completed
ADASYN Random Forest Selection Completed
ADASYN Forward feature selection Completed


#### 4. Variations of SMOTE

In [28]:
from imblearn.over_sampling import BorderlineSMOTE,SVMSMOTE,KMeansSMOTE

In [29]:
all_dataset["X_borderline_smote"],all_dataset["y_borderline_smote"] = BorderlineSMOTE().fit_resample(X_train,y_train)
all_dataset["X_svm_smote"],all_dataset["y_svm_smote"] = SVMSMOTE().fit_resample(X_train,y_train)

# X_kmeans_smote,y_kmeans_smote = KMeansSMOTE().fit_resample(X_train,y_train)


# all_dataset["X_kmeans_smote"],all_dataset["y_kmeans_smote"]= X_kmeans_smote,y_kmeans_smote

training_testing_function(all_dataset["X_borderline_smote"],all_dataset["y_borderline_smote"],"Borderline SMOTE")
training_testing_function(all_dataset["X_svm_smote"],all_dataset["y_svm_smote"],"SVM SMOTE")

# training_testing_function(all_dataset["X_kmeans_smote"],all_dataset["y_kmeans_smote"],"KMeans SMOTE")



Borderline SMOTE RCF Completed
Borderline SMOTE Random Forest Selection Completed
Borderline SMOTE Forward feature selection Completed
SVM SMOTE RCF Completed
SVM SMOTE Random Forest Selection Completed
SVM SMOTE Forward feature selection Completed


In [30]:
all_results

[('Cluster centroids RCF',
  0.9212984094659598,
  0.036949516648764766,
  0.8994624122862425),
 ('Cluster centroids Random Forest  Selection',
  0.9923106632491837,
  0.2819672131147541,
  0.9350297308005926),
 ('Cluster centroids Forward feature selection',
  0.9918015519118009,
  0.26917057902973396,
  0.9347747364280545),
 ('Near Miss RCF',
  0.5980653769179453,
  0.007456539645380847,
  0.7375673644528155),
 ('Near Miss Random Forest  Selection',
  0.6635300726800323,
  0.008996897621509825,
  0.7754493715043699),
 ('Near Miss Forward feature selection',
  0.6825778589234929,
  0.009857072449482503,
  0.8002694219218356),
 ('One Sided Selection RCF',
  0.9993679997191109,
  0.8085106382978723,
  0.8876320013092462),
 ('One Sided Selection Random Forest  Selection',
  0.9993153330290369,
  0.7914438502673796,
  0.8774191267671952),
 ('One Sided Selection Forward feature selection',
  0.9993679997191109,
  0.8064516129032258,
  0.8825387534023176),
 ('Tomek Links RCF',
  0.999367999

### Combination of Undersampling and Oversampling

#### 1. SMOTETomek

In [5]:
from imblearn.combine import SMOTETomek

In [6]:
all_dataset['X_smote_tomek'],all_dataset['y_smote_tomek'] = SMOTETomek().fit_resample(X_train,y_train)
training_testing_function(all_dataset['X_smote_tomek'],all_dataset['y_smote_tomek'],"SMOTE Tomek")

SMOTE Tomek RCF Completed
SMOTE Tomek Random Forest Selection Completed
SMOTE Tomek Forward feature selection Completed


In [7]:
all_results

[('SMOTE Tomek RCF',
  0.9976651100733822,
  0.5128205128205129,
  0.8562196016560818),
 ('SMOTE Tomek Random Forest  Selection',
  0.9991046662687406,
  0.767123287671233,
  0.9282460909237077),
 ('SMOTE Tomek Forward feature selection',
  0.9966468873986166,
  0.4558404558404559,
  0.9066420919802924)]

In [2]:
dataset1_results = [('Random Undersample RCF',
  0.9735086548927355,
  0.1001788908765653,
  0.9154260290216255),
 ('Random Undersample Random Forest  Selection',
  0.9836031038236017,
  0.1539855072463768,
  0.9255751998323245),
 ('Random Undersample Forward feature selection',
  0.9614128717390541,
  0.06785411365564037,
  0.8889947228187842),
  ('IHT RCF', 0.9990344440153085, 0.7417840375586854, 0.9027446797514729),
 ('IHT Random Forest  Selection',
  0.9990344440153085,
  0.7417840375586854,
  0.9027446797514729),
 ('IHT Forward feature selection',
  0.9991046662687406,
  0.7582938388625592,
  0.907873099295993),
 ('Cluster centroids RCF',
  0.9212984094659598,
  0.036949516648764766,
  0.8994624122862425),
 ('Cluster centroids Random Forest  Selection',
  0.9923106632491837,
  0.2819672131147541,
  0.9350297308005926),
 ('Cluster centroids Forward feature selection',
  0.9918015519118009,
  0.26917057902973396,
  0.9347747364280545),
 ('Near Miss RCF',
  0.5980653769179453,
  0.007456539645380847,
  0.7375673644528155),
 ('Near Miss Random Forest  Selection',
  0.6635300726800323,
  0.008996897621509825,
  0.7754493715043699),
 ('Near Miss Forward feature selection',
  0.6825778589234929,
  0.009857072449482503,
  0.8002694219218356),
 ('One Sided Selection RCF',
  0.9993679997191109,
  0.8085106382978723,
  0.8876320013092462),
 ('One Sided Selection Random Forest  Selection',
  0.9993153330290369,
  0.7914438502673796,
  0.8774191267671952),
 ('One Sided Selection Forward feature selection',
  0.9993679997191109,
  0.8064516129032258,
  0.8825387534023176),
 ('Tomek Links RCF',
  0.9993679997191109,
  0.8085106382978723,
  0.8876320013092462),
 ('Tomek Links Random Forest  Selection',
  0.9992802219023208,
  0.783068783068783,
  0.8774015409483996),
 ('Tomek Links Forward feature selection',
  0.9993679997191109,
  0.8064516129032258,
  0.8825387534023176),
 ('Random Oversampled RCF',
  0.9991222218320986,
  0.7395833333333335,
  0.8620426610430328),
 ('Random Oversampled Random Forest  Selection',
  0.9992802219023208,
  0.7939698492462313,
  0.902867780483043),
 ('Random Oversampled Forward feature selection',
  0.9990519995786665,
  0.7428571428571428,
  0.8976602247539421),
 ('SMOTE RCF', 0.9978757768336786, 0.5468164794007491, 0.871604860289642),
 ('SMOTE Random Forest  Selection',
  0.9989642217618764,
  0.7377777777777779,
  0.9230824997415963),
 ('SMOTE Forward feature selection',
  0.9963835539482462,
  0.4371584699453552,
  0.9065101983393244),
 ('ADASYN RCF', 0.9984375548611355, 0.6454183266932271, 0.9126322166458029),
 ('ADASYN Random Forest  Selection',
  0.9985955549313578,
  0.6721311475409836,
  0.9178046007373124),
 ('ADASYN Forward feature selection',
  0.9948562199360977,
  0.3644251626898048,
  0.9261182068494253),
 ('Borderline SMOTE RCF',
  0.9991748885221726,
  0.7614213197969544,
  0.8824420313989412),
 ('Borderline SMOTE Random Forest  Selection',
  0.9992275552122467,
  0.7864077669902914,
  0.9130278975687067),
 ('Borderline SMOTE Forward feature selection',
  0.9982795547909132,
  0.6230769230769231,
  0.9125530804612222),
 ('SVM SMOTE RCF', 0.9989291106351603, 0.6995073891625616, 0.8619459390396564),
 ('SVM SMOTE Random Forest  Selection',
  0.9992099996488887,
  0.7804878048780487,
  0.9079258567523802),
 ('SVM SMOTE Forward feature selection',
  0.9983322214809873,
  0.6215139442231076,
  0.8972997154686297),
  ('SMOTE Tomek RCF',
  0.9976651100733822,
  0.5128205128205129,
  0.8562196016560818),
 ('SMOTE Tomek Random Forest  Selection',
  0.9991046662687406,
  0.767123287671233,
  0.9282460909237077),
 ('SMOTE Tomek Forward feature selection',
  0.9966468873986166,
  0.4558404558404559,
  0.9066420919802924)]

### Top 3 dataset-1 results wrt f1 score and auroc score

In [17]:
sorted(dataset1_results, key=lambda x: (x[2],x[3]), reverse=True)[:3]

[('One Sided Selection RCF',
  0.9993679997191109,
  0.8085106382978723,
  0.8876320013092462),
 ('Tomek Links RCF',
  0.9993679997191109,
  0.8085106382978723,
  0.8876320013092462),
 ('One Sided Selection Forward feature selection',
  0.9993679997191109,
  0.8064516129032258,
  0.8825387534023176)]

### Trying Xgboost on dataset 1

In [10]:
# count examples in each class
counter = Counter(y_train)
# estimate scale_pos_weight value
estimate = counter[0] / counter[1]
print('Estimate: %.3f' % estimate)

Estimate: 577.287


In [10]:
# define model
model = XGBClassifier(scale_pos_weight=500)

In [12]:
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(model, X_train, y_train, scoring='f1', cv=cv, n_jobs=-1)
# summarize performance
print('Mean f1 score: %.5f' % np.mean(scores))

Mean f1 score: 0.87181


In [14]:
model.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=0, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=500, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [15]:
f1 = metrics.f1_score(y_test.values.ravel(),model.predict(X_test))
f1

0.8369565217391304

## Working with different dataset

In [77]:
dataset2 = pd.read_csv("dataset2.csv")
dataset2.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [78]:
dataset2.shape

(6362620, 11)

In [79]:
dataset2.isnull().values.any()

False

In [80]:
dataset2.drop("isFlaggedFraud",axis=1,inplace=True)


In [81]:
all_zero=list(dataset2.loc[dataset2["isFraud"]==0].index)

In [82]:
sample_zero = np.random.choice(all_zero,200000,replace=False)

In [83]:
all_one=list(dataset2.loc[dataset2["isFraud"]==1].index)

In [84]:
selected_sample = np.concatenate([sample_zero,all_one])

In [85]:
len(selected_sample)

208213

In [86]:
new_dataset2 = dataset2.loc[selected_sample]

In [87]:
new_dataset2.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud
4161929,304,CASH_OUT,110640.92,C793420685,98786.01,0.00,C698799929,504958.73,615599.65,0
4858807,348,CASH_OUT,225320.50,C274955916,361.00,0.00,C374387,0.00,225320.50,0
3265393,251,CASH_IN,336206.16,C1221366808,532555.99,868762.15,C1100157475,397962.98,61756.82,0
1818226,163,CASH_IN,141319.47,C240061710,1012540.83,1153860.30,C2006455995,597072.37,355816.92,0
3275504,251,CASH_OUT,132456.88,C81835133,0.00,0.00,C2038478708,1929924.77,2062381.65,0


In [88]:
y=new_dataset2['isFraud']
new_dataset2.drop('isFraud',axis=1,inplace=True)
X=new_dataset2

Since the destination account balances being zero is a strong indicator of fraud, we do not impute the account balance (before the transaction is made) with a statistic or from a distribution with a subsequent adjustment for the amount transacted. Doing so would mask this indicator of fraud and make fraudulent transactions appear genuine. Instead, below we replace the value of 0 with -1 which will be more useful to a suitable machine-learning (ML) algorithm detecting fraud.

In [89]:
X.loc[(X.oldbalanceDest == 0) & (X.newbalanceDest == 0) & (X.amount != 0), ['oldbalanceDest', 'newbalanceDest']] = - 1

The data also has several transactions with zero balances in the originating account both before and after a non-zero amount is transacted. In this case, the fraction of such transactions is much smaller in fraudulent (0.3%) compared to genuine transactions (47%). Once again, from similar reasoning as above, instead of imputing a numerical value we replace the value of 0 with a null value.

In [90]:
# X.loc[(X.oldbalanceOrg == 0) & (X.newbalanceOrig == 0) & (X.amount != 0),['oldbalanceOrg', 'newbalanceOrig']] = np.nan

Motivated by the possibility of zero-balances serving to differentiate between fraudulent and genuine transactions, we create 2 new features (columns) recording errors in the originating and destination accounts for each transaction. These new features turn out to be important in obtaining the best performance from the ML algorithm that we will finally use.

In [91]:
X['errorbalanceOrg'] = X.newbalanceOrig + X.amount - X.oldbalanceOrg
X['errorbalanceDest'] = X.oldbalanceDest + X.amount - X.newbalanceDest

In [92]:
# log transformed amount
X['ln_amount'] = np.log1p(X['amount'])

In [93]:
# correcting few negative errors
X['errorbalanceOrg'] = X['errorbalanceOrg'].apply(lambda x: 0 if x < 0 else x)

# log transformed origin account balance error
X['ln_errorBalanceOrig'] = np.log1p(X['errorbalanceOrg'])

In [94]:
X['errorbalanceDest'] = X['errorbalanceDest'].apply(lambda x: 0 if x < 0 else x)

# log transformed origin account balance error
X['ln_errorBalanceDest'] = np.log1p(X['errorbalanceDest'])

In [95]:
X["type"].value_counts()

CASH_OUT    74210
PAYMENT     67857
CASH_IN     44065
TRANSFER    20743
DEBIT        1338
Name: type, dtype: int64

In [96]:
X["type_transfer"]=X["type"].replace({'CASH_OUT':0,'PAYMENT':1,'CASH_IN':2,'TRANSFER':3,'DEBIT':4})

In [97]:
X.drop("type",axis=1,inplace=True)

In [98]:
X.head()

,step,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,errorbalanceOrg,errorbalanceDest,ln_amount,ln_errorBalanceOrig,ln_errorBalanceDest,type_transfer
4161929,304,110640.92,C793420685,98786.01,0.00,C698799929,504958.73,615599.65,11854.91,0.00,11.614054,9.380582,0.000000,0
4858807,348,225320.50,C274955916,361.00,0.00,C374387,0.00,225320.50,224959.50,0.00,12.325284,12.323680,0.000000,0
3265393,251,336206.16,C1221366808,532555.99,868762.15,C1100157475,397962.98,61756.82,672412.32,672412.32,12.725483,13.418628,13.418628,2
1818226,163,141319.47,C240061710,1012540.83,1153860.30,C2006455995,597072.37,355816.92,282638.94,382574.92,11.858785,12.551929,12.854682,2
3275504,251,132456.88,C81835133,0.00,0.00,C2038478708,1929924.77,2062381.65,132456.88,0.00,11.794020,11.794020,0.000000,0


In [99]:
X.drop(["amount","errorbalanceOrg","errorbalanceDest"],axis=1,inplace=True)


In [100]:
X.head()

,step,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,ln_amount,ln_errorBalanceOrig,ln_errorBalanceDest,type_transfer
4161929,304,C793420685,98786.01,0.00,C698799929,504958.73,615599.65,11.614054,9.380582,0.000000,0
4858807,348,C274955916,361.00,0.00,C374387,0.00,225320.50,12.325284,12.323680,0.000000,0
3265393,251,C1221366808,532555.99,868762.15,C1100157475,397962.98,61756.82,12.725483,13.418628,13.418628,2
1818226,163,C240061710,1012540.83,1153860.30,C2006455995,597072.37,355816.92,11.858785,12.551929,12.854682,2
3275504,251,C81835133,0.00,0.00,C2038478708,1929924.77,2062381.65,11.794020,11.794020,0.000000,0


In [101]:
X["type_name"]=X.step

In [102]:
X.loc[(X.nameOrig.str.contains("C")) & (X.nameDest.str.contains("C")),"type_name"]=0
X.loc[(X.nameOrig.str.contains("C")) & (X.nameDest.str.contains("M")),"type_name"]=1
X.loc[(X.nameOrig.str.contains("M")) & (X.nameDest.str.contains("C")),"type_name"]=2
X.loc[(X.nameOrig.str.contains("M")) & (X.nameDest.str.contains("M")),"type_name"]=3

In [103]:
X.tail()

,step,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,ln_amount,ln_errorBalanceOrig,ln_errorBalanceDest,type_transfer,type_name
6362615,743,C786484425,339682.13,0.0,C776919290,0.00,339682.13,12.735768,0.0,0.000000e+00,0,0
6362616,743,C1529008245,6311409.28,0.0,C1881841831,-1.00,-1.00,15.657870,0.0,1.565787e+01,3,0
6362617,743,C1162922333,6311409.28,0.0,C1365125890,68488.84,6379898.11,15.657870,0.0,9.950331e-03,0,0
6362618,743,C1685995037,850002.52,0.0,C2080388513,-1.00,-1.00,13.652996,0.0,1.365300e+01,3,0
6362619,743,C1280323807,850002.52,0.0,C873221189,6510099.11,7360101.63,13.652996,0.0,9.313226e-10,0,0


In [104]:
X.drop(["nameOrig","nameDest"],axis=1,inplace=True)

In [105]:
X.head()

,step,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,ln_amount,ln_errorBalanceOrig,ln_errorBalanceDest,type_transfer,type_name
4161929,304,98786.01,0.00,504958.73,615599.65,11.614054,9.380582,0.000000,0,0
4858807,348,361.00,0.00,0.00,225320.50,12.325284,12.323680,0.000000,0,0
3265393,251,532555.99,868762.15,397962.98,61756.82,12.725483,13.418628,13.418628,2,0
1818226,163,1012540.83,1153860.30,597072.37,355816.92,11.858785,12.551929,12.854682,2,0
3275504,251,0.00,0.00,1929924.77,2062381.65,11.794020,11.794020,0.000000,0,0


In [106]:
#first proceed without step column
step_col = X.step
X.drop("step",axis=1,inplace=True)


In [107]:
X.head()

,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,ln_amount,ln_errorBalanceOrig,ln_errorBalanceDest,type_transfer,type_name
4161929,98786.01,0.00,504958.73,615599.65,11.614054,9.380582,0.000000,0,0
4858807,361.00,0.00,0.00,225320.50,12.325284,12.323680,0.000000,0,0
3265393,532555.99,868762.15,397962.98,61756.82,12.725483,13.418628,13.418628,2,0
1818226,1012540.83,1153860.30,597072.37,355816.92,11.858785,12.551929,12.854682,2,0
3275504,0.00,0.00,1929924.77,2062381.65,11.794020,11.794020,0.000000,0,0


In [108]:
# dividing into training and testing

In [109]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0, stratify = y)

In [110]:
X.isnull().values.any()

False

In [111]:
# ss = StandardScaler()
# ss.fit(X_train.loc[:,"oldbalanceOrg"].values.reshape(-1,1))
# X_train.loc[:,"scaled_oldbalanceOrg"]= ss.transform(X_train.loc[:,"oldbalanceOrg"].values.reshape(-1,1))
# X_test.loc[:,"scaled_oldbalanceOrg"] = ss.transform(X_test.loc[:,"oldbalanceOrg"].values.reshape(-1,1))

# ss = StandardScaler()
# ss.fit(X_train.loc[:,"newbalanceOrig"].values.reshape(-1,1))
# X_train.loc[:,"scaled_newbalanceOrig"]= ss.transform(X_train.loc[:,"newbalanceOrig"].values.reshape(-1,1))
# X_test.loc[:,"scaled_newbalanceOrig"] = ss.transform(X_test.loc[:,"newbalanceOrig"].values.reshape(-1,1))

# ss = StandardScaler()
# ss.fit(X_train.loc[:,"oldbalanceDest"].values.reshape(-1,1))
# X_train.loc[:,"scaled_oldbalanceDest"]= ss.transform(X_train.loc[:,"oldbalanceDest"].values.reshape(-1,1))
# X_test.loc[:,"scaled_oldbalanceDest"] = ss.transform(X_test.loc[:,"oldbalanceDest"].values.reshape(-1,1))

# ss = StandardScaler()
# ss.fit(X_train.loc[:,"newbalanceDest"].values.reshape(-1,1))
# X_train.loc[:,"scaled_newbalanceDest"]= ss.transform(X_train.loc[:,"newbalanceDest"].values.reshape(-1,1))
# X_test.loc[:,"scaled_newbalanceDest"] = ss.transform(X_test.loc[:,"newbalanceDest"].values.reshape(-1,1))


cols=["oldbalanceOrg","newbalanceOrig","oldbalanceDest","newbalanceDest","ln_amount","ln_errorBalanceOrig","ln_errorBalanceDest"]
for c in cols:
    ss = StandardScaler()
    ss.fit(X_train.loc[:,str(c)].values.reshape(-1,1))
    transformed1 = ss.transform(X_train.loc[:,str(c)].values.reshape(-1,1))
    print(transformed1.shape)
    X_train["scaled_"+str(c)] = transformed1
    X_train.drop(c,axis=1,inplace=True)
    transformed2 = ss.transform(X_test.loc[:,str(c)].values.reshape(-1,1))
    X_test["scaled_"+str(c)] = transformed2
    X_test.drop(c,axis=1,inplace=True)
    


















(166570, 1)


C:\Users\Anuj\Miniconda3\envs\condaenv\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Anuj\Miniconda3\envs\condaenv\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
C:\Users\Anuj\Miniconda3\envs\condaenv\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#ind

(166570, 1)


C:\Users\Anuj\Miniconda3\envs\condaenv\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Anuj\Miniconda3\envs\condaenv\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
C:\Users\Anuj\Miniconda3\envs\condaenv\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#ind

(166570, 1)


...

C:\Users\Anuj\Miniconda3\envs\condaenv\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Anuj\Miniconda3\envs\condaenv\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
C:\Users\Anuj\Miniconda3\envs\condaenv\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#ind

(166570, 1)


C:\Users\Anuj\Miniconda3\envs\condaenv\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
C:\Users\Anuj\Miniconda3\envs\condaenv\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Anuj\Miniconda3\envs\condaenv\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


(166570, 1)


C:\Users\Anuj\Miniconda3\envs\condaenv\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Anuj\Miniconda3\envs\condaenv\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
C:\Users\Anuj\Miniconda3\envs\condaenv\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#ind

In [112]:
X_train.head()

,type_transfer,type_name,scaled_oldbalanceOrg,scaled_newbalanceOrig,scaled_oldbalanceDest,scaled_newbalanceDest,scaled_ln_amount,scaled_ln_errorBalanceOrig,scaled_ln_errorBalanceDest
1734096,2,0,1.531459,1.595803,0.787259,0.659175,0.366901,0.724860,1.197198
3757394,0,0,-0.296472,-0.286848,-0.241548,-0.251851,-0.096175,0.420943,-1.095397
4392239,2,0,0.928221,1.025453,0.534181,0.394472,0.744890,0.862541,1.328905
2113067,2,0,1.293792,1.351220,-0.220898,-0.271012,0.295542,0.698868,1.172333
4259067,0,0,-0.296472,-0.286848,-0.234673,-0.191515,0.800985,0.747727,-1.097254


In [42]:
X_train.to_csv("X_train_dataset2_preprocessed.csv",index=False)
X_test.to_csv("X_test_dataset2_preprocessed.csv",index=False)

In [43]:
y_train.to_csv("y_train_dataset2_preprocessed.csv",index=False)
y_test.to_csv("y_test_dataset2_preprocessed.csv",index=False)

C:\Users\Anuj\Miniconda3\envs\condaenv\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.
C:\Users\Anuj\Miniconda3\envs\condaenv\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  


In [52]:
X_train = pd.read_csv("X_train_dataset2_preprocessed.csv")
X_test = pd.read_csv("X_test_dataset2_preprocessed.csv")

y_train = pd.read_csv("y_train_dataset2_preprocessed.csv",header=None)
y_test = pd.read_csv("y_test_dataset2_preprocessed.csv",header=None)


#### 1. Random Undersample

In [94]:
undersample = RandomUnderSampler(sampling_strategy='majority')
# fit and apply the transform
all_dataset["X_train_undersample_dataset2"], all_dataset["y_train_undersample_dataset2"] = undersample.fit_resample(X_train, y_train)
# summarize class distribution
print(Counter(all_dataset["y_train_undersample_dataset2"]))

Counter({0: 6570, 1: 6570})


In [95]:
training_testing_function(all_dataset["X_train_undersample_dataset2"],all_dataset["y_train_undersample_dataset2"],"Random Undersample dataset 2")

Random Undersample dataset 2 RCF Completed
Random Undersample dataset 2 Random Forest Selection Completed
Random Undersample dataset 2 Forward feature selection Completed


In [96]:
all_results

[('SMOTE Tomek RCF',
  0.9976651100733822,
  0.5128205128205129,
  0.8562196016560818),
 ('SMOTE Tomek Random Forest  Selection',
  0.9991046662687406,
  0.767123287671233,
  0.9282460909237077),
 ('SMOTE Tomek Forward feature selection',
  0.9966468873986166,
  0.4558404558404559,
  0.9066420919802924),
 ('Random Undersample dataset 2 RCF',
  0.9634032130249982,
  0.6817042606516291,
  0.9777399650030433),
 ('Random Undersample dataset 2 Random Forest  Selection',
  0.9647719904905987,
  0.690048594971477,
  0.9787442863664029),
 ('Random Undersample dataset 2 RCF',
  0.9638354585404509,
  0.6845412651864264,
  0.9785486077297626),
 ('Random Undersample dataset 2 Random Forest  Selection',
  0.9659006315587254,
  0.697098976109215,
  0.9796236077297628),
 ('Random Undersample dataset 2 Forward feature selection',
  0.974233364551065,
  0.7525939589578049,
  0.9833774650030432)]

#### 2. Instance Hardness Threshold

In [97]:
from imblearn.under_sampling import InstanceHardnessThreshold

In [98]:
iht = InstanceHardnessThreshold(sampling_strategy='majority', random_state=42)
all_dataset["X_train_res_dataset2"], all_dataset["y_train_res_dataset2"] = iht.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(all_dataset["y_train_res_dataset2"]))

Resampled dataset shape Counter({0: 158482, 1: 6570})


In [99]:
training_testing_function(all_dataset["X_train_res_dataset2"],all_dataset["y_train_res_dataset2"],"IHT dataset 2")

IHT dataset 2 RCF Completed
IHT dataset 2 Random Forest Selection Completed
IHT dataset 2 Forward feature selection Completed


In [100]:
all_results

[('SMOTE Tomek RCF',
  0.9976651100733822,
  0.5128205128205129,
  0.8562196016560818),
 ('SMOTE Tomek Random Forest  Selection',
  0.9991046662687406,
  0.767123287671233,
  0.9282460909237077),
 ('SMOTE Tomek Forward feature selection',
  0.9966468873986166,
  0.4558404558404559,
  0.9066420919802924),
 ('Random Undersample dataset 2 RCF',
  0.9634032130249982,
  0.6817042606516291,
  0.9777399650030433),
 ('Random Undersample dataset 2 Random Forest  Selection',
  0.9647719904905987,
  0.690048594971477,
  0.9787442863664029),
 ('Random Undersample dataset 2 RCF',
  0.9638354585404509,
  0.6845412651864264,
  0.9785486077297626),
 ('Random Undersample dataset 2 Random Forest  Selection',
  0.9659006315587254,
  0.697098976109215,
  0.9796236077297628),
 ('Random Undersample dataset 2 Forward feature selection',
  0.974233364551065,
  0.7525939589578049,
  0.9833774650030432),
 ('IHT dataset 2 RCF',
  0.9962058449199145,
  0.9535840188014102,
  0.992188572732806),
 ('IHT dataset 2 Ra

#### 3. Cluster Centroid

In [58]:
from imblearn.under_sampling import ClusterCentroids

In [59]:
all_dataset["X_cluster_centroids_dataset2"],all_dataset["y_cluster_centroids_dataset2"]  = ClusterCentroids().fit_resample(X_train,y_train)


KeyboardInterrupt



In [ ]:
training_testing_function(all_dataset["X_cluster_centroids_dataset2"],all_dataset["y_cluster_centroids_dataset2"],"Cluster centroids dataset 2")

In [ ]:
all_results

#### 4. Near Miss

In [60]:
from imblearn.under_sampling import NearMiss

In [69]:
all_dataset['X_near_miss_dataset2'],all_dataset['y_near_miss_dataset2'] = NearMiss().fit_resample(X_train,y_train)
training_testing_function(all_dataset['X_near_miss_dataset2'],all_dataset['y_near_miss_dataset2'],"Near Miss dataset 2")

Near Miss dataset 2 RCF Completed
Near Miss dataset 2 Random Forest Selection Completed


C:\Users\Anuj\Miniconda3\envs\condaenv\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\Anuj\Miniconda3\envs\condaenv\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:115: UserWarning: Features [1] are constant.
  UserWarning)
C:\Users\Anuj\Miniconda3\envs\condaenv\lib\site-packages\sklearn\feature_selection\_univariate_selection.py:116: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw


Near Miss dataset 2 Forward feature selection Completed


In [67]:
all_dataset['y_near_miss_dataset2'].values

array([[0],
       [0],
       [0],
       ...,
       [1],
       [1],
       [1]], dtype=int64)

In [ ]:
all_results

#### 5. One Sided Selection

In [70]:
from imblearn.under_sampling import OneSidedSelection

In [71]:
all_dataset['X_one_sided_selection_dataset2'],all_dataset['y_one_sided_selection_dataset2'] = OneSidedSelection().fit_resample(X_train,y_train)
training_testing_function(all_dataset['X_one_sided_selection_dataset2'],all_dataset['y_one_sided_selection_dataset2'],"One Sided Selection dataset 2")

One Sided Selection dataset 2 RCF Completed
One Sided Selection dataset 2 Random Forest Selection Completed


C:\Users\Anuj\Miniconda3\envs\condaenv\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


One Sided Selection dataset 2 Forward feature selection Completed


In [72]:
all_results

[('Near Miss dataset 2 RCF',
  0.4844511682635737,
  0.12095156205216395,
  0.6831951536822884),
 ('Near Miss dataset 2 Random Forest  Selection',
  0.33410176980524936,
  0.09609492144207575,
  0.6040571895922093),
 ('Near Miss dataset 2 Forward feature selection',
  0.3651514059986072,
  0.10392163508795717,
  0.6374371500304321),
 ('Near Miss dataset 2 RCF',
  0.4844511682635737,
  0.12095156205216395,
  0.6831951536822884),
 ('Near Miss dataset 2 Random Forest  Selection',
  0.331844487668996,
  0.09580137787599116,
  0.6028821895922094),
 ('Near Miss dataset 2 Forward feature selection',
  0.3651514059986072,
  0.10392163508795717,
  0.6374371500304321),
 ('One Sided Selection dataset 2 RCF',
  0.9965420358763778,
  0.9573712255772646,
  0.9906126445526475),
 ('One Sided Selection dataset 2 Random Forest  Selection',
  0.9981749633791994,
  0.977245508982036,
  0.9958399650030433),
 ('One Sided Selection dataset 2 Forward feature selection',
  0.9970943495905674,
  0.9642540620384

#### 6. Tomek Links

In [73]:
from imblearn.under_sampling import TomekLinks

In [74]:
all_dataset['X_tomek_links_dataset2'],all_dataset['y_tomek_links_dataset2'] = TomekLinks().fit_resample(X_train,y_train)


In [75]:
training_testing_function(all_dataset['X_tomek_links_dataset2'],all_dataset['y_tomek_links_dataset2'],"Tomek Links dataset 2")

Tomek Links dataset 2 RCF Completed
Tomek Links dataset 2 Random Forest Selection Completed


C:\Users\Anuj\Miniconda3\envs\condaenv\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


Tomek Links dataset 2 Forward feature selection Completed


In [76]:
all_results

[('Near Miss dataset 2 RCF',
  0.4844511682635737,
  0.12095156205216395,
  0.6831951536822884),
 ('Near Miss dataset 2 Random Forest  Selection',
  0.33410176980524936,
  0.09609492144207575,
  0.6040571895922093),
 ('Near Miss dataset 2 Forward feature selection',
  0.3651514059986072,
  0.10392163508795717,
  0.6374371500304321),
 ('Near Miss dataset 2 RCF',
  0.4844511682635737,
  0.12095156205216395,
  0.6831951536822884),
 ('Near Miss dataset 2 Random Forest  Selection',
  0.331844487668996,
  0.09580137787599116,
  0.6028821895922094),
 ('Near Miss dataset 2 Forward feature selection',
  0.3651514059986072,
  0.10392163508795717,
  0.6374371500304321),
 ('One Sided Selection dataset 2 RCF',
  0.9965420358763778,
  0.9573712255772646,
  0.9906126445526475),
 ('One Sided Selection dataset 2 Random Forest  Selection',
  0.9981749633791994,
  0.977245508982036,
  0.9958399650030433),
 ('One Sided Selection dataset 2 Forward feature selection',
  0.9970943495905674,
  0.9642540620384

### Oversampling data

#### 1. Random Oversampling

In [113]:
ros = RandomOverSampler(random_state=0)
all_dataset["X_train_random_oversampled_dataset2"], all_dataset["y_train_random_oversampled_dataset2"] = ros.fit_resample(X_train, y_train)

print(sorted(Counter(all_dataset["y_train_random_oversampled_dataset2"]).items()))

[(0, 160000), (1, 160000)]


In [114]:
training_testing_function(all_dataset["X_train_random_oversampled_dataset2"],all_dataset["y_train_random_oversampled_dataset2"],"Random Oversampled dataset2")

Random Oversampled dataset2 RCF Completed
Random Oversampled dataset2 Random Forest Selection Completed
Random Oversampled dataset2 Forward feature selection Completed


In [115]:
all_results

[('Near Miss dataset 2 RCF',
  0.4844511682635737,
  0.12095156205216395,
  0.6831951536822884),
 ('Near Miss dataset 2 Random Forest  Selection',
  0.33410176980524936,
  0.09609492144207575,
  0.6040571895922093),
 ('Near Miss dataset 2 Forward feature selection',
  0.3651514059986072,
  0.10392163508795717,
  0.6374371500304321),
 ('Near Miss dataset 2 RCF',
  0.4844511682635737,
  0.12095156205216395,
  0.6831951536822884),
 ('Near Miss dataset 2 Random Forest  Selection',
  0.331844487668996,
  0.09580137787599116,
  0.6028821895922094),
 ('Near Miss dataset 2 Forward feature selection',
  0.3651514059986072,
  0.10392163508795717,
  0.6374371500304321),
 ('One Sided Selection dataset 2 RCF',
  0.9965420358763778,
  0.9573712255772646,
  0.9906126445526475),
 ('One Sided Selection dataset 2 Random Forest  Selection',
  0.9981749633791994,
  0.977245508982036,
  0.9958399650030433),
 ('One Sided Selection dataset 2 Forward feature selection',
  0.9970943495905674,
  0.9642540620384

#### 2. SMOTE

In [116]:
from imblearn.over_sampling import SMOTE, ADASYN
all_dataset["X_smote_dataset2"], all_dataset["y_smote_dataset2"] = SMOTE().fit_resample(X_train, y_train)
print(sorted(Counter(all_dataset["y_smote_dataset2"]).items()))



[(0, 160000), (1, 160000)]


In [117]:
training_testing_function(all_dataset['X_smote_dataset2'],all_dataset['y_smote_dataset2'],"SMOTE dataset2")

SMOTE dataset2 RCF Completed
SMOTE dataset2 Random Forest Selection Completed
SMOTE dataset2 Forward feature selection Completed


In [118]:
all_results

[('Near Miss dataset 2 RCF',
  0.4844511682635737,
  0.12095156205216395,
  0.6831951536822884),
 ('Near Miss dataset 2 Random Forest  Selection',
  0.33410176980524936,
  0.09609492144207575,
  0.6040571895922093),
 ('Near Miss dataset 2 Forward feature selection',
  0.3651514059986072,
  0.10392163508795717,
  0.6374371500304321),
 ('Near Miss dataset 2 RCF',
  0.4844511682635737,
  0.12095156205216395,
  0.6831951536822884),
 ('Near Miss dataset 2 Random Forest  Selection',
  0.331844487668996,
  0.09580137787599116,
  0.6028821895922094),
 ('Near Miss dataset 2 Forward feature selection',
  0.3651514059986072,
  0.10392163508795717,
  0.6374371500304321),
 ('One Sided Selection dataset 2 RCF',
  0.9965420358763778,
  0.9573712255772646,
  0.9906126445526475),
 ('One Sided Selection dataset 2 Random Forest  Selection',
  0.9981749633791994,
  0.977245508982036,
  0.9958399650030433),
 ('One Sided Selection dataset 2 Forward feature selection',
  0.9970943495905674,
  0.9642540620384

#### 3. ADASYN

In [119]:
all_dataset['X_adasyn_dataset2'],all_dataset['y_adasyn_dataset2'] = ADASYN().fit_resample(X_train, y_train)
print(sorted(Counter(all_dataset["y_adasyn_dataset2"]).items()))

[(0, 160000), (1, 160212)]


In [120]:
training_testing_function(all_dataset['X_adasyn_dataset2'],all_dataset['y_adasyn_dataset2'],"ADASYN dataset2")

ADASYN dataset2 RCF Completed
ADASYN dataset2 Random Forest Selection Completed
ADASYN dataset2 Forward feature selection Completed


In [121]:
all_results

[('Near Miss dataset 2 RCF',
  0.4844511682635737,
  0.12095156205216395,
  0.6831951536822884),
 ('Near Miss dataset 2 Random Forest  Selection',
  0.33410176980524936,
  0.09609492144207575,
  0.6040571895922093),
 ('Near Miss dataset 2 Forward feature selection',
  0.3651514059986072,
  0.10392163508795717,
  0.6374371500304321),
 ('Near Miss dataset 2 RCF',
  0.4844511682635737,
  0.12095156205216395,
  0.6831951536822884),
 ('Near Miss dataset 2 Random Forest  Selection',
  0.331844487668996,
  0.09580137787599116,
  0.6028821895922094),
 ('Near Miss dataset 2 Forward feature selection',
  0.3651514059986072,
  0.10392163508795717,
  0.6374371500304321),
 ('One Sided Selection dataset 2 RCF',
  0.9965420358763778,
  0.9573712255772646,
  0.9906126445526475),
 ('One Sided Selection dataset 2 Random Forest  Selection',
  0.9981749633791994,
  0.977245508982036,
  0.9958399650030433),
 ('One Sided Selection dataset 2 Forward feature selection',
  0.9970943495905674,
  0.9642540620384

#### 4. Variations of SMOTE

In [122]:
from imblearn.over_sampling import BorderlineSMOTE,SVMSMOTE,KMeansSMOTE

In [123]:
all_dataset["X_borderline_smote_dataset2"],all_dataset["y_borderline_smote_dataset2"] = BorderlineSMOTE().fit_resample(X_train,y_train)
all_dataset["X_svm_smote_dataset2"],all_dataset["y_svm_smote_dataset2"] = SVMSMOTE().fit_resample(X_train,y_train)

# X_kmeans_smote,y_kmeans_smote = KMeansSMOTE().fit_resample(X_train,y_train)


# all_dataset["X_kmeans_smote"],all_dataset["y_kmeans_smote"]= X_kmeans_smote,y_kmeans_smote

training_testing_function(all_dataset["X_borderline_smote_dataset2"],all_dataset["y_borderline_smote_dataset2"],"Borderline SMOTE dataset2")
training_testing_function(all_dataset["X_svm_smote_dataset2"],all_dataset["y_svm_smote_dataset2"],"SVM SMOTE dataset2")

# training_testing_function(all_dataset["X_kmeans_smote"],all_dataset["y_kmeans_smote"],"KMeans SMOTE")



Borderline SMOTE dataset2 RCF Completed
Borderline SMOTE dataset2 Random Forest Selection Completed
Borderline SMOTE dataset2 Forward feature selection Completed
SVM SMOTE dataset2 RCF Completed
SVM SMOTE dataset2 Random Forest Selection Completed
SVM SMOTE dataset2 Forward feature selection Completed


In [124]:
all_results

[('Near Miss dataset 2 RCF',
  0.4844511682635737,
  0.12095156205216395,
  0.6831951536822884),
 ('Near Miss dataset 2 Random Forest  Selection',
  0.33410176980524936,
  0.09609492144207575,
  0.6040571895922093),
 ('Near Miss dataset 2 Forward feature selection',
  0.3651514059986072,
  0.10392163508795717,
  0.6374371500304321),
 ('Near Miss dataset 2 RCF',
  0.4844511682635737,
  0.12095156205216395,
  0.6831951536822884),
 ('Near Miss dataset 2 Random Forest  Selection',
  0.331844487668996,
  0.09580137787599116,
  0.6028821895922094),
 ('Near Miss dataset 2 Forward feature selection',
  0.3651514059986072,
  0.10392163508795717,
  0.6374371500304321),
 ('One Sided Selection dataset 2 RCF',
  0.9965420358763778,
  0.9573712255772646,
  0.9906126445526475),
 ('One Sided Selection dataset 2 Random Forest  Selection',
  0.9981749633791994,
  0.977245508982036,
  0.9958399650030433),
 ('One Sided Selection dataset 2 Forward feature selection',
  0.9970943495905674,
  0.9642540620384

### Combination of Undersampling and Oversampling

#### 1. SMOTETomek

In [125]:
from imblearn.combine import SMOTETomek

In [126]:
all_dataset['X_smote_tomek_dataset2'],all_dataset['y_smote_tomek_dataset2'] = SMOTETomek().fit_resample(X_train,y_train)
training_testing_function(all_dataset['X_smote_tomek_dataset2'],all_dataset['y_smote_tomek_dataset2'],"SMOTE Tomek dataset2")

SMOTE Tomek dataset2 RCF Completed
SMOTE Tomek dataset2 Random Forest Selection Completed
SMOTE Tomek dataset2 Forward feature selection Completed


In [127]:
all_results

[('Near Miss dataset 2 RCF',
  0.4844511682635737,
  0.12095156205216395,
  0.6831951536822884),
 ('Near Miss dataset 2 Random Forest  Selection',
  0.33410176980524936,
  0.09609492144207575,
  0.6040571895922093),
 ('Near Miss dataset 2 Forward feature selection',
  0.3651514059986072,
  0.10392163508795717,
  0.6374371500304321),
 ('Near Miss dataset 2 RCF',
  0.4844511682635737,
  0.12095156205216395,
  0.6831951536822884),
 ('Near Miss dataset 2 Random Forest  Selection',
  0.331844487668996,
  0.09580137787599116,
  0.6028821895922094),
 ('Near Miss dataset 2 Forward feature selection',
  0.3651514059986072,
  0.10392163508795717,
  0.6374371500304321),
 ('One Sided Selection dataset 2 RCF',
  0.9965420358763778,
  0.9573712255772646,
  0.9906126445526475),
 ('One Sided Selection dataset 2 Random Forest  Selection',
  0.9981749633791994,
  0.977245508982036,
  0.9958399650030433),
 ('One Sided Selection dataset 2 Forward feature selection',
  0.9970943495905674,
  0.9642540620384

In [10]:
dataset2_results = [
 ('Random Undersample dataset 2 RCF',
  0.9638354585404509,
  0.6845412651864264,
  0.9785486077297626),
 ('Random Undersample dataset 2 Random Forest  Selection',
  0.9659006315587254,
  0.697098976109215,
  0.9796236077297628),
 ('Random Undersample dataset 2 Forward feature selection',
  0.974233364551065,
  0.7525939589578049,
  0.9833774650030432),
 ('IHT dataset 2 RCF',
  0.9962058449199145,
  0.9535840188014102,
  0.992188572732806),
 ('IHT dataset 2 Random Forest  Selection',
  0.997574622385515,
  0.9700029700029701,
  0.9958192863664029),
 ('IHT dataset 2 Forward feature selection',
  0.9967821722738516,
  0.9604252805670409,
  0.9933640368228851),
  ('Near Miss dataset 2 RCF',
  0.4844511682635737,
  0.12095156205216395,
  0.6831951536822884),
 ('Near Miss dataset 2 Random Forest  Selection',
  0.331844487668996,
  0.09580137787599116,
  0.6028821895922094),
 ('Near Miss dataset 2 Forward feature selection',
  0.3651514059986072,
  0.10392163508795717,
  0.6374371500304321),
 ('One Sided Selection dataset 2 RCF',
  0.9965420358763778,
  0.9573712255772646,
  0.9906126445526475),
 ('One Sided Selection dataset 2 Random Forest  Selection',
  0.9981749633791994,
  0.977245508982036,
  0.9958399650030433),
 ('One Sided Selection dataset 2 Forward feature selection',
  0.9970943495905674,
  0.9642540620384046,
  0.9952774650030433),
 ('Tomek Links dataset 2 RCF',
  0.9965660495161252,
  0.9576797869192069,
  0.9909169659160073),
 ('Tomek Links dataset 2 Random Forest  Selection',
  0.9981749633791994,
  0.977245508982036,
  0.9958399650030433),
 ('Tomek Links dataset 2 Forward feature selection',
  0.9963739403981462,
  0.9554177738411573,
  0.9908169659160073),
 ('Random Oversampled dataset2 RCF',
  0.9864082799029849,
  0.8520648196549923,
  0.9891313222763238),
 ('Random Oversampled dataset2 Random Forest  Selection',
  0.9879691664865644,
  0.8671439936356405,
  0.9914029290931223),
 ('Random Oversampled dataset2 Forward feature selection',
  0.9933962490694714,
  0.9224696926980546,
  0.9945197504564819),
 ('SMOTE dataset2 RCF',
  0.9829983430588575,
  0.8218419728233518,
  0.988231786366403),
 ('SMOTE dataset2 Random Forest  Selection',
  0.9855197752323319,
  0.8443870967741934,
  0.990419750456482),
 ('SMOTE dataset2 Forward feature selection',
  0.9925077443988185,
  0.9129464285714285,
  0.9940572504564821),
 ('ADASYN dataset2 RCF', 0.9802127608481618, 0.798828125, 0.987657250456482),
 ('ADASYN dataset2 Random Forest  Selection',
  0.9818697019907308,
  0.8127944458219687,
  0.9893952145465611),
 ('ADASYN dataset2 Forward feature selection',
  0.9842950796052158,
  0.8335877862595421,
  0.9903658931832015),
 ('Borderline SMOTE dataset2 RCF',
  0.9832624930960786,
  0.8241231390360837,
  0.988369286366403),
 ('Borderline SMOTE dataset2 Random Forest  Selection',
  0.984631270561679,
  0.8364008179959099,
  0.989957250456482),
 ('Borderline SMOTE dataset2 Forward feature selection',
  0.9993276180870735,
  0.9915048543689321,
  0.9970236077297626),
 ('SVM SMOTE dataset2 RCF',
  0.9831184112575944,
  0.8228772990677753,
  0.9882942863664028),
 ('SVM SMOTE dataset2 Random Forest  Selection',
  0.9846792978411738,
  0.8367451381780963,
  0.9896904290931223),
 ('SVM SMOTE dataset2 Forward feature selection',
  0.9628749129505559,
  0.6787198669991686,
  0.977756786366403),
 ('SMOTE Tomek dataset2 RCF',
  0.9830223566986048,
  0.822227809906965,
  0.9888279290931223),
 ('SMOTE Tomek dataset2 Random Forest  Selection',
  0.9854477343130899,
  0.8437338834450747,
  0.990382250456482),
 ('SMOTE Tomek dataset2 Forward feature selection',
  0.9922676080013447,
  0.9104062326099054,
  0.993932250456482)] 

In [11]:
sorted(dataset2_results,key=lambda x: (x[2],x[3]), reverse=True)[:3]

[('Borderline SMOTE dataset2 Forward feature selection',
  0.9993276180870735,
  0.9915048543689321,
  0.9970236077297626),
 ('One Sided Selection dataset 2 Random Forest  Selection',
  0.9981749633791994,
  0.977245508982036,
  0.9958399650030433),
 ('Tomek Links dataset 2 Random Forest  Selection',
  0.9981749633791994,
  0.977245508982036,
  0.9958399650030433)]